In [0]:
from pyspark import pipelines as dp
from pyspark.sql.functions import col,countDistinct,sum

In [0]:
@dp.table(
    name = "gold_customer_summary",
    table_properties = {"quality": "gold"}
)
def create_customer_summary():
    df_cust = spark.read.table("live.silver_customers_py")
    df_add = spark.read.table("live.silver_addresses_py")
    df_orders = spark.read.table("live.silver_orders_clean_py")
    return(
        df_cust.alias("c").join(df_add.alias("a"),col("a.customer_id") == col("c.customer_id"))
                          .join(df_orders.alias("o"),col("o.customer_id") == col("c.customer_id"))
                          .where("a.__END_AT is null" )
                          .groupBy(col("c.customer_id"),col("c.customer_name"),col("c.date_of_birth"),col("c.telephone"),col("c.email"),col("a.address_line_1"),col("a.state"),col("a.city"),col("a.postcode"))
                          .agg(countDistinct(col("o.order_id")).alias("total_orders"),sum(col("o.quantity")).alias("total_items_ordered"),sum(col("o.quantity") * col("o.price")).alias("total_amount_ordered"))
    )